In [1]:
from torchvision import models
import torch

import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import matplotlib.pyplot as plt
from itertools import cycle


resnet34_pretrained = models.resnet34(pretrained=True)

print(resnet34_pretrained)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [2]:
num_classes= 18
num_ftrs = resnet34_pretrained.fc.in_features
resnet34_pretrained.fc = nn.Linear(num_ftrs, num_classes)

In [3]:
class TrainDataset(Dataset):
    def __init__(self, img_paths, transform, json_data):
        self.img_paths = img_paths
        self.transform = transform
        self.json_data = json_data
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        label = json_data[index//7]['img'][index%7]['class']
        
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [4]:
import json
with open("train.json", "r") as f:
    json_data = json.load(f)
json_data = json_data['train']

In [5]:
# image_paths = []
# for image in json_data:
#     for img in image['img']:
#         print(img['path'])
#         image_paths.append(img['path'])

In [6]:
# image_paths[:10]
path_dir = 'input/data/train/images'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = []
for image in json_data:
    for img in image['img']:
        image_paths.append(img['path'])

In [8]:
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    transforms.RandomHorizontalFlip(0.5),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
train_dataset = TrainDataset(image_paths, transform, json_data)

In [16]:
from sklearn.model_selection import StratifiedKFold

img = []
label = []
for i in train_dataset:
    img.append(list(i))
    label.append(i[1])

In [17]:
skfold = StratifiedKFold(n_splits=5)

In [ ]:
n_iter = 0
for train_index, test_index in skfold.split(df,df[1]):
    n_iter += 1
    label_train = df[1].iloc[train_index]
    label_test = df[1].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

In [42]:
for train_index, test_index in skfold.split(df,df[1]):
    train_index_list = train_index
    test_index_list = test_index
    break
    

In [25]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.img_paths = df
        
    def __getitem__(self, index):
        image = self.img_paths[0].iloc[index]
        label = self.img_paths[1].iloc[index]
        
        return image, label

    def __len__(self):
        return len(self.img_paths)

In [43]:
train_dataset = TrainDataset(df.iloc[train_index_list])

In [45]:
loader = DataLoader(train_dataset,batch_size=64,shuffle=True)

In [46]:
test_data = TrainDataset(df.iloc[test_index_list])
test_loader = DataLoader(test_data,batch_size=64,shuffle=False)

In [ ]:
# # meta 데이터와 이미지 경로를 불러옵니다.
# path_dir = 'input/data/train/images'

# # Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.

# transform = transforms.Compose([
#     Resize((512, 384), Image.BILINEAR),
#     transforms.RandomHorizontalFlip(0.5),
#     ToTensor(),
#     Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
# ])

# train_dataset = TrainDataset(image_paths[:18200], transform, json_data[:2600])
# test_dataset = TrainDataset(image_paths[18200:], transform, json_data[2600:])

# loader = DataLoader(
#     train_dataset,
#     batch_size=64, # 
#     shuffle=True # shuffle 추가
# )

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=64,
#     shuffle=False
# )

from sklearn.metrics import f1_score
device = torch.device('cuda')
model = resnet34_pretrained.to(device)
model.train()

import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    running_loss = 0.0
    n_iter = 0
    epoch_f1 = 0
    for (i, data) in enumerate(loader, 0):
        n_iter += 1
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        

        
        # t_ = torch.zeros((100,18))
        # t_[range(100),labels]=1
        # print(t_)
        # t_ = t_.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        epoch_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average='macro')
        if i % 100 == 0:
            epoch_f1_2 = 0
            print('{},{:.5f} loss: {:.3f}'.format(epoch+1, i+1, running_loss/100))
            with torch.no_grad():
                for (j, data) in enumerate(test_loader, 0):
                    val_inputs, val_labels = data
                    val_inputs = val_inputs.to(device)
                    val_labels = val_labels.to(device)
        
                    val_outputs = model(val_inputs)
                    _, val_preds = torch.max(val_outputs, 1)
                    epoch_f1_2 += f1_score(val_preds.cpu().numpy(), val_labels.cpu().numpy(), average='macro')
                print('train_f1:{:.4f}'.format(epoch_f1/n_iter))
                print('val_f1:{:.4f}'.format(epoch_f1_2/j))
            running_loss=0.0
                    
        


    print('Finished')
   

torch.save(model.state_dict(), '/opt/ml/test_resnet_base.pt')

1,1.00000 loss: 0.031
train_f1:0.0069
val_f1:0.2058
1,101.00000 loss: 0.629
train_f1:0.6176
val_f1:0.6525


In [ ]:
        test_inputs, test_labels = data2
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)
        test_outputs = model(test_inputs)
        _, test_preds = torch.max(test_outputs, 1)
        
        epoch_f1_2 += f1_score(test_preds.cpu().numpy(), test_labels.cpu().numpy(), average='macro')
        print('test_f1:{:.4f}'.format(epoch_f1_2))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'batch_200_RANDOM_epoch_50_resnet_34.csv'), index=False)

print('test inference is done!')